In [ ]:
# nse scratchpad initiation
from helper import *
from nse_func import *

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

ib =  get_connected('nse', 'live')

In [ ]:
%%time
with open(logpath+'build.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'build.log')
base(ib)
print('Base build completed')

In [ ]:
# ... make the input dataframes
#______________________________

df_ac = util.df(ib.accountSummary()) # account summary

# ..positions

df_p = util.df(ib.positions())

df_pc = util.df(list(df_p.contract))
df_pc = df_pc[list(df_pc)[:6]]
df_pc = df_pc.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

df_pc = df_pc.join(df_p[['position', 'avgCost']]) # merge contract with postion and avgCost
df_pos = df_pc.assign(avgCost=np.where(df_pc.secType == 'OPT', df_pc.avgCost, df_pc.avgCost))

df_buy = df_pos[df_pos.secType == 'OPT'] # filter out any assigned stock

# ..Get the buy option prices
buy_opts = ib.qualifyContracts(*[Contract(conId=c) for c in df_buy.conId])
buy_ticks = ib.reqTickers(*buy_opts)
buyOptPrices = {t.contract.conId: t.marketPrice() for t in buy_ticks} # {symbol: undPrice}

df_buy = df_buy.assign(optPrice = df_buy.conId.map(buyOptPrices))
df_buy = df_buy.assign(optPrice = df_buy.optPrice.fillna(prec)) # make NaN to lowest price (prec)

#..get harvest price

# dte
df_buy = df_buy.assign(dte=[get_dte(d) for d in df_buy.expiration])

# first hvstPrice
df_buy = df_buy.assign(hvstPrice=[get_prec(min(hvstPricePct(d)*c,mp*0.9),prec) 
                              for d, c, mp in zip(df_buy.dte, df_buy.avgCost, df_buy.optPrice)])

# adjust to lower hvstPrice for hvstPrice = optPrice
df_buy = df_buy.assign(hvstPrice=np.where((df_buy.optPrice == df_buy.hvstPrice) &
             (df_buy.hvstPrice > prec), df_buy.optPrice - prec, df_buy.hvstPrice))

# reduce the buy price to even lesser for hvstPrice > 0.05
mask = (df_buy.hvstPrice > df_buy.optPrice*0.7) & (df_buy.hvstPrice != 0.05)
df_buy.loc[mask, 'hvstPrice'] = get_prec(df_buy[mask].hvstPrice*0.5, prec)

#...prepare to place the orders
buy_orders = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) 
                      for position, hvstPrice in zip(df_buy.position, df_buy.hvstPrice)]

#...prepare to place the orders
buy_orders = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) 
                      for position, hvstPrice in zip(df_buy.position, df_buy.hvstPrice)]

hco = list(zip(buy_opts, buy_orders))